In [458]:
include("packages.jl")
Random.seed!(1234) 
# x, myset, pfmin, prevminneg, prev, M, n = rand(101, 100), [1,4,5,6], rand(18,100), rand(101,100), rand(101,100), 20000, 100
# y, z, m, z_int, c, c3, c4 = zeros(n), rand(M,n), rand(1,n), rand([-1 1],M,n), Matrix{Float64}(undef,M,1), Matrix{Float64}(undef,M,1), Matrix{Float64}(undef,M,1)

# y = ((-1)^length(myset)) .* @avx(exp.(sum(log.(1e-50 .+ (prod(pfmin[myset, :],dims=1) .* prevminneg .+ (1 .- prev))), dims=2)))
# y = ((-1)^length(myset)) .* exp.(sum(@avx(log.(1e-50 .+ (prod(pfmin[myset, :],dims=1) .* prevminneg .+ (1 .- prev)))), dims=2));
# y = ((-1)^length(myset)) .* exp.(sum(@avx(log.(1e-50 .+ (@avx(prod(pfmin[myset, :],dims=1)) .* prevminneg .+ (1 .- prev)))), dims=2));
# y = @avx(exp.(x));
# y = ((-1)^length(myset)) .* exp.(sum(@avx(log.(1e-50 .+ @avx(prod(pfmin[myset, :],dims=1)) .* prevminneg .+ (1 .- prev))), dims=2)); 
# sum(@avx(log.(1e-50 .+ (prod(pfmin[myset, :],dims=1) .* prevminneg .+ (1 .- prev)))), dims=2)
# y = @avx(log.(1e-50 .+ (prod(pfmin[myset, :],dims=1) .* prevminneg .+ (1 .- prev))))

findfirst(isnan, y)
log1 = log.(1e-50 .+ (prod(pfmin[myset, :],dims=1) .* prevminneg .+ (1 .- prev)))
log1[98:101,66:70]
y[98:101,66:71]


4×6 Matrix{Float64}:
 -0.679357  -0.0644841   -0.801967   -0.289086   -0.919589  NaN
 -1.03028   -1.33012     -3.5148     -0.599392  NaN          -0.233989
 -0.972536  -2.10419     -0.994394  NaN          -1.76684    -0.755459
 -1.33382   -0.459411   NaN          -2.23007    -3.82388    -1.19842

In [482]:
zzz1 = zeros(101,1)
zzz2 = zeros(101,1)
for k=1:2^17;   a = 1e-50 .+ (prod(pfmin[myset, :],dims=1) .* prevminneg .+ (1 .- prev)); zzz1 += ((-1)^length(myset)) .* exp.(sum(@avx(log.(a)), dims=2)); end;
for k=1:2^17;   zzz2 += ((-1)^length(myset)) .* exp.(sum((log.(1e-50 .+ (prod(pfmin[myset, :],dims=1) .* prevminneg .+ (1 .- prev)))), dims=2)); end;


In [498]:
maximum((zzz1-zzz2))

4.484155085839415e-44

1×101 adjoint(::Matrix{Float64}) with eltype Float64:
 5.377e-30  1.30941e-28  1.43047e-33  …  1.68368e-40  1.36177e-37